In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import math
from datetime import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as  mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.svm import SVR
from itertools import combinations
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')
plt.rcParams["figure.figsize"] = (20,7)
import pickle

# Entrainement sur des données Journalieres

# 1er Scenario

Entrainement sur des données de Meknes et test par des données de Berkane

In [2]:
EnamMekjr= pd.read_excel(r'EnamMksJr.xlsx')
EnamBerkaneJr= pd.read_excel(r'AnamBerkaneJr.xlsx')

In [3]:
ytrain=EnamMekjr["ETP quotidien [mm]"].astype('int')
Xtrain=EnamMekjr.drop(columns=['ETP quotidien [mm]','Date/heure','Precipitation Somme','Unnamed: 0'])
ytest=EnamBerkaneJr['ETP quotidien [mm]']
Xtest=EnamBerkaneJr.drop(columns=['ETP quotidien [mm]','Date/heure','Precipitation Somme','Vitesse Max max','Unnamed: 0'])

# Entrainement sur 4 variables**

In [4]:
grid = { 
    'n_estimators': [200,300,400,500],
    'max_features': ['sqrt','log2'],
    'max_depth' : [3,4,5,6,7]
}

**Xgboost**

In [5]:
xgb_model = xgb.XGBRegressor(verbosity = 0)
clf = GridSearchCV(xgb_model,grid, cv=5,n_jobs=1)
clf.fit(Xtrain, ytrain)
modelxgb=clf.best_estimator_
ypred=modelxgb.predict(Xtest)

In [6]:
FS_Xgb_4vR2=r2_score(ytest,ypred)
FS_Xgb_4vMAE=mae(ytest,ypred)
FS_Xgb_4vMSE=mse(ytest,ypred)
FS_Xgb_4vRMSE=mse(ytest,ypred,squared=False)# ou bien on utilise np.sqrt(np.mean((ytest-ypred)**2))


In [7]:
print("R2 = ",FS_Xgb_4vR2)
print("Mae = ",FS_Xgb_4vMAE)
print("Mse = ",FS_Xgb_4vMSE)
print("Rmse =",FS_Xgb_4vRMSE)

R2 =  0.893945181977736
Mae =  0.3807799128676528
Mse =  0.22082200343118824
Rmse = 0.46991701760118054


**Rf**

In [8]:
## Grid Search function
clf = GridSearchCV(estimator=RandomForestRegressor(), param_grid=grid, cv= 5,n_jobs=2)
clf.fit(Xtrain, ytrain)
ypred=clf.predict(Xtest)

In [9]:
FS_RF_4vR2=r2_score(ytest,ypred)
FS_RF_4vMAE=mae(ytest,ypred)
FS_RF_4vMSE=mse(ytest,ypred)
FS_RF_4vRMSE=mse(ytest,ypred,squared=False)# ou bien on utilise np.sqrt(np.mean((ytest-ypred)**2))


In [10]:
print("R2 = ",FS_RF_4vR2)
print("Mae = ",FS_RF_4vMAE)
print("Mse = ",FS_RF_4vMSE)
print("Rmse =",FS_RF_4vRMSE)

R2 =  0.91166490141167
Mae =  0.3415655741557728
Mse =  0.18392689561233896
Rmse = 0.4288669905837228


**svr**

In [11]:
Svrparam = {
                'C': [1.1, 5.4, 170, 1001],

                'epsilon': [0.0003, 0.007, 0.0109, 0.019, 0.14, 0.05, 8, 0.2, 3, 2, 7],

                'gamma': [0.7001, 0.008, 0.001, 3.1, 1, 1.3, 5]
        }

In [12]:
svr = SVR()
clf = GridSearchCV(svr,Svrparam,cv=5,n_jobs=-1)
clf.fit(Xtrain,ytrain)
modelsvr=clf.best_estimator_
ypred=modelsvr.predict(Xtest)

In [13]:
FS_SVR_4vR2=r2_score(ytest,ypred)
FS_SVR_4vMAE=mae(ytest,ypred)
FS_SVR_4vMSE=mse(ytest,ypred)
FS_SVR_4vRMSE=mse(ytest,ypred,squared=False)# ou bien on utilise np.sqrt(np.mean((ytest-ypred)**2))

In [14]:
print("R2 = ",FS_SVR_4vR2)
print("Mae = ",FS_SVR_4vMAE)
print("Mse = ",FS_SVR_4vMSE)
print("Rmse =",FS_SVR_4vRMSE)

R2 =  0.894114237811271
Mae =  0.3581392606253651
Mse =  0.22047000388464164
Rmse = 0.4695423344967327


**Meilleur modele**

In [15]:
scoresR2=[FS_RF_4vR2,FS_Xgb_4vR2,FS_SVR_4vR2]
scoreMae=[FS_RF_4vMAE,FS_Xgb_4vMAE,FS_SVR_4vMAE]
scoreMse=[FS_RF_4vMSE,FS_Xgb_4vMSE,FS_SVR_4vMSE]
scoreRmse=[FS_RF_4vRMSE,FS_Xgb_4vRMSE,FS_SVR_4vRMSE]
Models=['Rf','Xgb','Svr']

In [16]:
tmp = max(scoresR2)
index = scoresR2.index(tmp)

In [17]:
print("Le meilleur model est:",Models[index])
print("R2=",scoresR2[index])
print("Mae=",scoreMae[index])
print("mse=",scoreMse[index])
print("Rmse=",scoreRmse[index])

Le meilleur model est: Rf
R2= 0.91166490141167
Mae= 0.3415655741557728
mse= 0.18392689561233896
Rmse= 0.4288669905837228


# Creation des combinaisons

In [20]:
temp=['Température min', 'Température max', 'Température moy']
hum=['Humidité min', 'Humidité max', 'Humidité moy']
VitVent=['Vitesse du vent  max', 'Vitesse du vent  moy']
Ray=['Rayonnement solaire moy']
#Création des combinaison de trois variables
comb3Var = combinations([VitVent, hum,Ray, temp], 3)
#Création des combinaison de deux variables
comb2Var = combinations([VitVent, hum,Ray, temp], 2)
#Création des combinaison d'une variables
comb1Var = combinations([VitVent, hum,Ray, temp], 1)

In [21]:
comb3v=list(comb3Var)
comb2v=list(comb2Var)
comb1v=list(comb1Var)

# Entrainement sur 3 variables

**RF**

In [26]:
FR_3v_rf_r2={}
FR_3v_rf_mae={}
FR_3v_rf_mse={}
FR_3v_rf_Rmse={}
for a in range(0,len(comb3v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in comb3v[a] :
        for x in i:
            xtr.insert(0,x,Xtrain[x])
            xts.insert(0,x,Xtest[x])
    modelrfc =RandomForestRegressor()

    grids = GridSearchCV(modelrfc,grid,cv=5,n_jobs=-1)

    grids.fit(xtr,ytrain)
    modelrfc=grids.best_estimator_
    modelrfc.fit(xtr,ytrain)
    ypred=modelrfc.predict(xts)
    scoreR2=r2_score(ytest,ypred)
    scoreMae=mae(ytest,ypred)
    scoreMse=mse(ytest,ypred)
    scoreRmse = mse(ytest,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    FR_3v_rf_r2[var]=scoreR2
    FR_3v_rf_mae[var]=scoreMae
    FR_3v_rf_mse[var]=scoreMse
    FR_3v_rf_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.6591836523115844
Mae 0.6477935939364694
Mse 0.7096306429271051
Rmse 0.8423957757058763
------
R2 0.721092049445338
Mae 0.6006309037596285
Mse 0.5807280947994075
Rmse 0.7620551783167722
------
R2 0.9181743632871276
Mae 0.33237588822638237
Mse 0.1703732217726861
Rmse 0.41276291230279655
------
R2 0.8847249294301671
Mae 0.3796633293651078
Mse 0.2400199491508089
Rmse 0.4899183086503391
------


In [28]:
FR_3v_rf_r2

{'Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,': 0.6591836523115844,
 'Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,': 0.721092049445338,
 'Température moy,Température max,Température min,Rayonnement solaire moy,Vitesse du vent  moy,Vitesse du vent  max,': 0.9181743632871276,
 'Température moy,Température max,Température min,Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,': 0.8847249294301671}

In [29]:
FR_3v_rf_r2['Rayonnement  & Humidité & Vitesse du vent'] = FR_3v_rf_r2.pop('Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,')
FR_3v_rf_r2['Température  & Humidité & Vitesse du vent'] = FR_3v_rf_r2.pop('Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,')
FR_3v_rf_r2['Température  & Rayonement & Vitesse du vent'] = FR_3v_rf_r2.pop('Température moy,Température max,Température min,Rayonnement solaire moy,Vitesse du vent  moy,Vitesse du vent  max,')
FR_3v_rf_r2['Température  & Rayonement &  Humidité'] = FR_3v_rf_r2.pop('Température moy,Température max,Température min,Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,')

In [33]:
Variables = list(FR_3v_rf_r2.keys())
ScoresR2 = list(FR_3v_rf_r2.values())
ScoreMae=list(FR_3v_rf_mae.values())
ScoreMse=list(FR_3v_rf_mse.values())
ScoreRmse=list(FR_3v_rf_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [34]:
print("La meilleur combinaison pour trois variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour trois variables est: Température  & Rayonement & Vitesse du vent
R2 0.9181743632871276
Mae 0.33237588822638237
Mse 0.1703732217726861
Rmse 0.41276291230279655


**Xgboost**

In [35]:
xgparam = {"learning_rate": (0.05, 0.10, 0.15),
                        "max_depth": [ 3, 4, 5, 6, 8],
                        "min_child_weight": [ 1, 3, 5, 7],
                        "gamma":[ 0.0, 0.1, 0.2],
                        "colsample_bytree":[ 0.3, 0.4]}

In [36]:
FR_3v_xgb_r2={}
FR_3v_xgb_mae={}
FR_3v_xgb_mse={}
FR_3v_xgb_Rmse={}

for a in range(0,len(comb3v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in comb3v[a] :
        
        for x in i:
            
            xtr.insert(0,x,Xtrain[x])
            xts.insert(0,x,Xtest[x])
    modelxgb =xgb.XGBRegressor(verbosity = 0)

    grids = GridSearchCV(modelxgb,xgparam,cv=5,n_jobs=-1)

    grids.fit(xtr,ytrain)
    modelxgb=grids.best_estimator_
    modelxgb.fit(xtr,ytrain)
    ypred=modelxgb.predict(xts)
    scoreR2=r2_score(ytest,ypred)
    scoreMae=mae(ytest,ypred)
    scoreMse=mse(ytest,ypred)
    scoreRmse = mse(ytest,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    FR_3v_xgb_r2[var]=scoreR2
    FR_3v_xgb_mae[var]=scoreMae
    FR_3v_xgb_mse[var]=scoreMse
    FR_3v_xgb_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.6530017900910583
Mae 0.6655236580821283
Mse 0.7225022052561789
Rmse 0.8500012972085271
------
R2 0.6578914380948475
Mae 0.6543618290436497
Mse 0.712321226320895
Rmse 0.8439912477750554
------
R2 0.9049750193179152
Mae 0.37338862303633236
Mse 0.1978562313484225
Rmse 0.444810331881379
------
R2 0.8623929509801607
Mae 0.407327940056341
Mse 0.2865184705181012
Rmse 0.535274201244653
------


In [37]:
FR_3v_xgb_r2

{'Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,': 0.6530017900910583,
 'Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,': 0.6578914380948475,
 'Température moy,Température max,Température min,Rayonnement solaire moy,Vitesse du vent  moy,Vitesse du vent  max,': 0.9049750193179152,
 'Température moy,Température max,Température min,Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,': 0.8623929509801607}

In [38]:
FR_3v_xgb_r2['Rayonnement  & Humidité & Vitesse du vent'] = FR_3v_xgb_r2.pop('Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,')
FR_3v_xgb_r2['Température  & Humidité & Vitesse du vent'] = FR_3v_xgb_r2.pop('Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,')
FR_3v_xgb_r2['Température  & Rayonement & Vitesse du vent'] = FR_3v_xgb_r2.pop('Température moy,Température max,Température min,Rayonnement solaire moy,Vitesse du vent  moy,Vitesse du vent  max,')
FR_3v_xgb_r2['Température  & Rayonement &  Humidité'] = FR_3v_xgb_r2.pop('Température moy,Température max,Température min,Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,')

In [39]:
Variables = list(FR_3v_xgb_r2.keys())
ScoresR2 = list(FR_3v_xgb_r2.values())
ScoreMae=list(FR_3v_xgb_mae.values())
ScoreMse=list(FR_3v_xgb_mse.values())
ScoreRmse=list(FR_3v_xgb_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [40]:
print("La meilleur combinaison pour trois variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour trois variables est: Température  & Rayonement & Vitesse du vent
R2 0.9049750193179152
Mae 0.37338862303633236
Mse 0.1978562313484225
Rmse 0.444810331881379


**SVR**

In [41]:
FR_3v_svr_r2={}
FR_3v_svr_mae={}
FR_3v_svr_mse={}
FR_3v_svr_Rmse={}

for a in range(0,len(comb3v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in comb3v[a] :
        
        for x in i:
            xtr.insert(0,x,Xtrain[x])
            xts.insert(0,x,Xtest[x])
    modelsvr = SVR()

    grids = GridSearchCV(modelsvr,Svrparam,cv=3,n_jobs=-1)

    grids.fit(xtr,ytrain)
    modelsvr=grids.best_estimator_
    modelsvr.fit(xtr,ytrain)
    ypred=modelsvr.predict(xts)
    scoreR2=r2_score(ytest,ypred)
    scoreMae=mae(ytest,ypred)
    scoreMse=mse(ytest,ypred)
    scoreRmse = mse(ytest,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    FR_3v_svr_r2[var]=scoreR2
    FR_3v_svr_mae[var]=scoreMae
    FR_3v_svr_mse[var]=scoreMse
    FR_3v_svr_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.6932069549292201
Mae 0.5938710472422103
Mse 0.6387890349032745
Rmse 0.7992427884587227
------
R2 0.2738419534798485
Mae 0.852919789833237
Mse 1.511969730659434
Rmse 1.2296217835820225
------
R2 0.9264905803459886
Mae 0.2923029014279074
Mse 0.15305761323974063
Rmse 0.39122578294348215
------
R2 0.8712503601934191
Mae 0.3932875987392367
Mse 0.2680760189241437
Rmse 0.5177605806974336
------


In [42]:
FR_3v_svr_r2['Rayonnement  & Humidité & Vitesse du vent'] = FR_3v_svr_r2.pop('Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,')
FR_3v_svr_r2['Température  & Humidité & Vitesse du vent'] = FR_3v_svr_r2.pop('Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,')
FR_3v_svr_r2['Température  & Rayonement & Vitesse du vent'] = FR_3v_svr_r2.pop('Température moy,Température max,Température min,Rayonnement solaire moy,Vitesse du vent  moy,Vitesse du vent  max,')
FR_3v_svr_r2['Température  & Rayonement &  Humidité'] = FR_3v_svr_r2.pop('Température moy,Température max,Température min,Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,')

In [43]:
Variables = list(FR_3v_svr_r2.keys())
ScoresR2 = list(FR_3v_svr_r2.values())
ScoreMae=list(FR_3v_svr_mae.values())
ScoreMse=list(FR_3v_svr_mse.values())
ScoreRmse=list(FR_3v_svr_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [44]:
print("La meilleur combinaison pour trois variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour trois variables est: Température  & Rayonement & Vitesse du vent
R2 0.9264905803459886
Mae 0.2923029014279074
Mse 0.15305761323974063
Rmse 0.39122578294348215


# Entrainement sur Deux variables

**RF**

In [45]:
FR_2v_rf_r2={}
FR_2v_rf_mae={}
FR_2v_rf_mse={}
FR_2v_rf_Rmse={}
for a in range(0,len(comb2v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in comb2v[a] :
        for x in i:
            xtr.insert(0,x,Xtrain[x])
            xts.insert(0,x,Xtest[x])
    modelrfc =RandomForestRegressor()

    grids = GridSearchCV(modelrfc,grid,cv=5,n_jobs=-1)

    grids.fit(xtr,ytrain)
    modelrfc=grids.best_estimator_
    modelrfc.fit(xtr,ytrain)
    ypred=modelrfc.predict(xts)
    scoreR2=r2_score(ytest,ypred)
    scoreMae=mae(ytest,ypred)
    scoreMse=mse(ytest,ypred)
    scoreRmse = mse(ytest,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    FR_2v_rf_r2[var]=scoreR2
    FR_2v_rf_mae[var]=scoreMae
    FR_2v_rf_mse[var]=scoreMse
    FR_2v_rf_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 -0.0659678059803055
Mae 1.183047070206237
Mse 2.21950450624793
Rmse 1.4898001564800327
------
R2 0.767283818595275
Mae 0.5373924017243832
Mse 0.48454991830601407
Rmse 0.696096199031437
------
R2 0.7035084303154628
Mae 0.6013036096326818
Mse 0.6173398214162489
Rmse 0.7857097564726105
------
R2 0.6958887850922175
Mae 0.6046647393513812
Mse 0.6332050631375509
Rmse 0.7957418319640804
------
R2 0.6944609565014008
Mae 0.6333508360937106
Mse 0.6361780159544072
Rmse 0.7976076829835625
------
R2 0.9077711299548104
Mae 0.31830569112713386
Mse 0.192034310532672
Rmse 0.4382171956149964
------


In [46]:
FR_2v_rf_r2

{'Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,': -0.0659678059803055,
 'Rayonnement solaire moy,Vitesse du vent  moy,Vitesse du vent  max,': 0.767283818595275,
 'Température moy,Température max,Température min,Vitesse du vent  moy,Vitesse du vent  max,': 0.7035084303154628,
 'Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,': 0.6958887850922175,
 'Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,': 0.6944609565014008,
 'Température moy,Température max,Température min,Rayonnement solaire moy,': 0.9077711299548104}

In [47]:
FR_2v_rf_r2['Humidité & Vitesse du vent'] = FR_2v_rf_r2.pop('Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,')
FR_2v_rf_r2['Rayonnement & Vitesse du vent'] = FR_2v_rf_r2.pop('Rayonnement solaire moy,Vitesse du vent  moy,Vitesse du vent  max,')
FR_2v_rf_r2['Température & Vitesse du vent'] = FR_2v_rf_r2.pop('Température moy,Température max,Température min,Vitesse du vent  moy,Vitesse du vent  max,')
FR_2v_rf_r2['Rayonnement & Humidité'] = FR_2v_rf_r2.pop('Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,')
FR_2v_rf_r2['Température & Humidité'] = FR_2v_rf_r2.pop('Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,')
FR_2v_rf_r2['Température & Rayonnement'] = FR_2v_rf_r2.pop('Température moy,Température max,Température min,Rayonnement solaire moy,')

In [48]:
Variables = list(FR_2v_rf_r2.keys())
ScoresR2 = list(FR_2v_rf_r2.values())
ScoreMae=list(FR_2v_rf_mae.values())
ScoreMse=list(FR_2v_rf_mse.values())
ScoreRmse=list(FR_2v_rf_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [49]:
print("La meilleur combinaison pour deux variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour deux variables est: Température & Rayonnement
R2 0.9077711299548104
Mae 0.31830569112713386
Mse 0.192034310532672
Rmse 0.4382171956149964


**xgboost**

In [50]:
FR_2v_xgb_r2={}
FR_2v_xgb_mae={}
FR_2v_xgb_mse={}
FR_2v_xgb_Rmse={}

for a in range(0,len(comb2v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in comb2v[a] :
        
        for x in i:
            
            xtr.insert(0,x,Xtrain[x])
            xts.insert(0,x,Xtest[x])
    modelxgb =xgb.XGBRegressor(verbosity = 0)

    grids = GridSearchCV(modelxgb,xgparam,cv=5,n_jobs=-1)

    grids.fit(xtr,ytrain)
    modelxgb=grids.best_estimator_
    modelxgb.fit(xtr,ytrain)
    ypred=modelxgb.predict(xts)
    scoreR2=r2_score(ytest,ypred)
    scoreMae=mae(ytest,ypred)
    scoreMse=mse(ytest,ypred)
    scoreRmse = mse(ytest,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    FR_2v_xgb_r2[var]=scoreR2
    FR_2v_xgb_mae[var]=scoreMae
    FR_2v_xgb_mse[var]=scoreMse
    FR_2v_xgb_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 -0.13281784877056046
Mae 1.2278586643095877
Mse 2.358696300205898
Rmse 1.5358047728164859
------
R2 0.8019465608154435
Mae 0.5078022500740619
Mse 0.41237690133030647
Rmse 0.6421657895982209
------
R2 0.6186310516138276
Mae 0.6952884295320437
Mse 0.794067226737411
Rmse 0.8911044982141045
------
R2 0.7058064622581922
Mae 0.6092345305604256
Mse 0.6125549749848889
Rmse 0.7826589135663689
------
R2 0.6076968439114786
Mae 0.6978323001818252
Mse 0.8168338836021521
Rmse 0.9037886277234031
------
R2 0.8923259579853637
Mae 0.35212099221604537
Mse 0.22419347011858035
Rmse 0.47349072865113245
------


In [51]:
FR_2v_xgb_r2['Humidité & Vitesse du vent'] = FR_2v_xgb_r2.pop('Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,')
FR_2v_xgb_r2['Rayonnement & Vitesse du vent'] = FR_2v_xgb_r2.pop('Rayonnement solaire moy,Vitesse du vent  moy,Vitesse du vent  max,')
FR_2v_xgb_r2['Température & Vitesse du vent'] = FR_2v_xgb_r2.pop('Température moy,Température max,Température min,Vitesse du vent  moy,Vitesse du vent  max,')
FR_2v_xgb_r2['Rayonnement & Humidité'] = FR_2v_xgb_r2.pop('Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,')
FR_2v_xgb_r2['Température & Humidité'] = FR_2v_xgb_r2.pop('Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,')
FR_2v_xgb_r2['Température & Rayonnement'] = FR_2v_xgb_r2.pop('Température moy,Température max,Température min,Rayonnement solaire moy,')

In [52]:
Variables = list(FR_2v_xgb_r2.keys())
ScoresR2 = list(FR_2v_xgb_r2.values())
ScoreMae=list(FR_2v_xgb_mae.values())
ScoreMse=list(FR_2v_xgb_mse.values())
ScoreRmse=list(FR_2v_xgb_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [53]:
print("La meilleur combinaison pour deux variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour deux variables est: Température & Rayonnement
R2 0.8923259579853637
Mae 0.35212099221604537
Mse 0.22419347011858035
Rmse 0.47349072865113245


**svr**

In [72]:
FS_2v_svr_r2={}
FS_2v_svr_mae={}
FS_2v_svr_mse={}
FS_2v_svr_Rmse={}

for a in range(0,len(comb2v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in comb2v[a] :
        
        for x in i:
            xtr.insert(0,x,Xtrain[x])
            xts.insert(0,x,Xtest[x])
    modelsvr = SVR()

    grids = GridSearchCV(modelsvr,Svrparam,cv=3,n_jobs=-1)

    grids.fit(xtr,ytrain)
    modelsvr=grids.best_estimator_
    modelsvr.fit(xtr,ytrain)
    ypred=modelsvr.predict(xts)
    scoreR2=r2_score(ytest,ypred)
    scoreMae=mae(ytest,ypred)
    scoreMse=mse(ytest,ypred)
    scoreRmse = mse(ytest,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    FS_2v_svr_r2[var]=scoreR2
    FS_2v_svr_mae[var]=scoreMae
    FS_2v_svr_mse[var]=scoreMse
    FS_2v_svr_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 -1.3287179228990489
Mae 1.380912645216147
Mse 4.848739234578959
Rmse 2.201985293906151
------
R2 0.8251491999178517
Mae 0.45128899797054495
Mse 0.3640655341804516
Rmse 0.603378433638833
------
R2 0.5318470512386002
Mae 0.768076155860916
Mse 0.9747645037305948
Rmse 0.9873016275336503
------
R2 0.6960222116053909
Mae 0.6030053726822698
Mse 0.6329272491683298
Rmse 0.7955672499344916
------
R2 0.6899254845385971
Mae 0.6277981286721604
Mse 0.6456215473658895
Rmse 0.803505785521106
------
R2 0.888783113645397
Mae 0.3499911263480399
Mse 0.23157020226131098
Rmse 0.48121741683080316
------


In [73]:
FS_2v_svr_r2

{'Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,': -1.3287179228990489,
 'Rayonnement solaire moy,Vitesse du vent  moy,Vitesse du vent  max,': 0.8251491999178517,
 'Température moy,Température max,Température min,Vitesse du vent  moy,Vitesse du vent  max,': 0.5318470512386002,
 'Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,': 0.6960222116053909,
 'Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,': 0.6899254845385971,
 'Température moy,Température max,Température min,Rayonnement solaire moy,': 0.888783113645397}

In [74]:
FS_2v_svr_r2['Humidité & Vitesse du vent'] = FS_2v_svr_r2.pop('Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,')
FS_2v_svr_r2['Rayonnement & Vitesse du vent'] = FS_2v_svr_r2.pop('Rayonnement solaire moy,Vitesse du vent  moy,Vitesse du vent  max,')
FS_2v_svr_r2['Température & Vitesse du vent'] = FS_2v_svr_r2.pop('Température moy,Température max,Température min,Vitesse du vent  moy,Vitesse du vent  max,')
FS_2v_svr_r2['Rayonnement & Humidité'] = FS_2v_svr_r2.pop('Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,')
FS_2v_svr_r2['Température & Humidité'] = FS_2v_svr_r2.pop('Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,')
FS_2v_svr_r2['Température & Rayonnement'] = FS_2v_svr_r2.pop('Température moy,Température max,Température min,Rayonnement solaire moy,')

In [75]:
Variables = list(FS_2v_svr_r2.keys())
ScoresR2 = list(FS_2v_svr_r2.values())
ScoreMae=list(FS_2v_svr_mae.values())
ScoreMse=list(FS_2v_svr_mse.values())
ScoreRmse=list(FS_2v_svr_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [76]:
print("La meilleur combinaison pour deux variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour deux variables est: Température & Rayonnement
R2 0.888783113645397
Mae 0.3499911263480399
Mse 0.23157020226131098
Rmse 0.48121741683080316


# Entrainement sur une variable

**RF**

In [109]:
FR_1v_rf_r2={}
FR_1v_rf_mae={}
FR_1v_rf_mse={}
FR_1v_rf_Rmse={}
for a in range(0,len(comb1v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in comb1v[a] :
        for x in i:
            xtr.insert(0,x,Xtrain[x])
            xts.insert(0,x,Xtest[x])
    modelrfc =RandomForestRegressor()

    grids = GridSearchCV(modelrfc,grid,cv=5,n_jobs=-1)

    grids.fit(xtr,ytrain)
    modelrfc=grids.best_estimator_
    modelrfc.fit(xtr,ytrain)
    ypred=modelrfc.predict(xts)
    scoreR2=r2_score(ytest,ypred)
    scoreMae=mae(ytest,ypred)
    scoreMse=mse(ytest,ypred)
    scoreRmse = mse(ytest,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    FR_1v_rf_r2[var]=scoreR2
    FR_1v_rf_mae[var]=scoreMae
    FR_1v_rf_mse[var]=scoreMse
    FR_1v_rf_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 -0.03961104978753083
Mae 1.159812265341746
Mse 2.1646257952664634
Rmse 1.4712667315162344
------
R2 -0.027343734677958587
Mae 1.19559644837085
Mse 2.1390834092652096
Rmse 1.4625605660160572
------
R2 0.7890540941044986
Mae 0.4956519901094757
Mse 0.4392209465266605
Rmse 0.6627374642546326
------
R2 0.719081484247466
Mae 0.607534338138414
Mse 0.584914392445309
Rmse 0.7647969615821633
------


In [110]:
FR_1v_rf_r2

{'Vitesse du vent  moy,Vitesse du vent  max,': -0.03961104978753083,
 'Humidité moy,Humidité max,Humidité min,': -0.027343734677958587,
 'Rayonnement solaire moy,': 0.7890540941044986,
 'Température moy,Température max,Température min,': 0.719081484247466}

In [111]:
FR_1v_rf_r2['Vitesse du vent'] = FR_1v_rf_r2.pop('Vitesse du vent  moy,Vitesse du vent  max,')
FR_1v_rf_r2['Humidité'] = FR_1v_rf_r2.pop('Humidité moy,Humidité max,Humidité min,')
FR_1v_rf_r2['Rayonnement'] = FR_1v_rf_r2.pop( 'Rayonnement solaire moy,')
FR_1v_rf_r2['Température'] = FR_1v_rf_r2.pop('Température moy,Température max,Température min,')

In [112]:
Variables = list(FR_1v_rf_r2.keys())
ScoresR2 = list(FR_1v_rf_r2.values())
ScoreMae=list(FR_1v_rf_mae.values())
ScoreMse=list(FR_1v_rf_mse.values())
ScoreRmse=list(FR_1v_rf_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [108]:
ScoreMse

[2.1758011532135555,
 2.134090210483256,
 0.43787612199442755,
 0.5846563624115616]

In [106]:
ScoresR2

[0.719205409073109,
 -0.024945636753201494,
 -0.044978271056229424,
 0.7896999768463526]

In [113]:
print("La meilleur combinaison pour Une variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour Une variables est: Rayonnement
R2 0.7890540941044986
Mae 0.4956519901094757
Mse 0.4392209465266605
Rmse 0.6627374642546326


**Xgboost**

In [114]:
FR_1v_xgb_r2={}
FR_1v_xgb_mae={}
FR_1v_xgb_mse={}
FR_1v_xgb_Rmse={}

for a in range(0,len(comb1v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in comb1v[a] :
        
        for x in i:
            
            xtr.insert(0,x,Xtrain[x])
            xts.insert(0,x,Xtest[x])
    modelxgb =xgb.XGBRegressor(verbosity = 0)

    grids = GridSearchCV(modelxgb,xgparam,cv=5,n_jobs=-1)

    grids.fit(xtr,ytrain)
    modelxgb=grids.best_estimator_
    modelxgb.fit(xtr,ytrain)
    ypred=modelxgb.predict(xts)
    scoreR2=r2_score(ytest,ypred)
    scoreMae=mae(ytest,ypred)
    scoreMse=mse(ytest,ypred)
    scoreRmse = mse(ytest,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    FR_1v_xgb_r2[var]=scoreR2
    FR_1v_xgb_mae[var]=scoreMae
    FR_1v_xgb_mse[var]=scoreMse
    FR_1v_xgb_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.12327835201422532
Mae 1.0800742650906192
Mse 1.8254656824649826
Rmse 1.3510979544300195
------
R2 -0.06619098871063755
Mae 1.2281711338712826
Mse 2.2199692060943135
Rmse 1.489956108781166
------
R2 0.7867892739735294
Mae 0.5007067267999953
Mse 0.44393664099541064
Rmse 0.6662857052311798
------
R2 0.6325736603174088
Mae 0.6848127758325037
Mse 0.7650366287467042
Rmse 0.8746637232369388
------


In [115]:
FR_1v_xgb_r2['Vitesse du vent'] = FR_1v_xgb_r2.pop('Vitesse du vent  moy,Vitesse du vent  max,')
FR_1v_xgb_r2['Humidité'] = FR_1v_xgb_r2.pop('Humidité moy,Humidité max,Humidité min,')
FR_1v_xgb_r2['Rayonnement'] = FR_1v_xgb_r2.pop( 'Rayonnement solaire moy,')
FR_1v_xgb_r2['Température'] = FR_1v_xgb_r2.pop('Température moy,Température max,Température min,')

In [116]:
Variables = list(FR_1v_xgb_r2.keys())
ScoresR2 = list(FR_1v_xgb_r2.values())
ScoreMae=list(FR_1v_xgb_mae.values())
ScoreMse=list(FR_1v_xgb_mse.values())
ScoreRmse=list(FR_1v_xgb_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [117]:
print("La meilleur combinaison pour Une variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour Une variables est: Rayonnement
R2 0.7867892739735294
Mae 0.5007067267999953
Mse 0.44393664099541064
Rmse 0.6662857052311798


**Svr**

In [118]:
FS_1v_svr_r2={}
FS_1v_svr_mae={}
FS_1v_svr_mse={}
FS_1v_svr_Rmse={}

for a in range(0,len(comb1v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in comb1v[a] :
        
        for x in i:
            xtr.insert(0,x,Xtrain[x])
            xts.insert(0,x,Xtest[x])
    modelsvr = SVR()

    grids = GridSearchCV(modelsvr,Svrparam,cv=3,n_jobs=-1)

    grids.fit(xtr,ytrain)
    modelsvr=grids.best_estimator_
    modelsvr.fit(xtr,ytrain)
    ypred=modelsvr.predict(xts)
    scoreR2=r2_score(ytest,ypred)
    scoreMae=mae(ytest,ypred)
    scoreMse=mse(ytest,ypred)
    scoreRmse = mse(ytest,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    FS_1v_svr_r2[var]=scoreR2
    FS_1v_svr_mae[var]=scoreMae
    FS_1v_svr_mse[var]=scoreMse
    FS_1v_svr_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 -0.36949298664907126
Mae 1.3795017266828582
Mse 2.8514893583931644
Rmse 1.6886353538858425
------
R2 -0.02078174766415475
Mae 1.14511572079302
Mse 2.1254203702265357
Rmse 1.4578821523794492
------
R2 0.7833829867955779
Mae 0.5029852147744986
Mse 0.4510290406894942
Rmse 0.6715869569084068
------
R2 0.6678188733756101
Mae 0.6592493335906789
Mse 0.6916508203128316
Rmse 0.8316554697185798
------


In [119]:
FS_1v_svr_r2['Vitesse du vent'] = FS_1v_svr_r2.pop('Vitesse du vent  moy,Vitesse du vent  max,')
FS_1v_svr_r2['Humidité'] = FS_1v_svr_r2.pop('Humidité moy,Humidité max,Humidité min,')
FS_1v_svr_r2['Rayonnement'] = FS_1v_svr_r2.pop( 'Rayonnement solaire moy,')
FS_1v_svr_r2['Température'] = FS_1v_svr_r2.pop('Température moy,Température max,Température min,')

In [120]:
Variables = list(FS_1v_svr_r2.keys())
ScoresR2 = list(FS_1v_svr_r2.values())
ScoreMae=list(FS_1v_svr_mae.values())
ScoreMse=list(FS_1v_svr_mse.values())
ScoreRmse=list(FS_1v_svr_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [121]:
print("La meilleur combinaison pour Une variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour Une variables est: Rayonnement
R2 0.7833829867955779
Mae 0.5029852147744986
Mse 0.4510290406894942
Rmse 0.6715869569084068


# 2éme Scénario

**Entrainemet , validation et test sur des données mélanger**

In [125]:
data=pd.concat([EnamMekjr,EnamBerkaneJr],axis=0,join='inner')

In [126]:
data=data.drop(columns=['Date/heure','Precipitation Somme','Unnamed: 0'])

In [127]:
y=data.iloc[:,-1]
X=data.drop(columns=['ETP quotidien [mm]'])

In [128]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrainement sur 4 variables**

In [129]:
grid = { 
    'n_estimators': [200,300,400,500],
    'max_features': ['sqrt','log2'],
    'max_depth' : [3,4,5,6,7]
}

**Xgboost**

In [130]:
xgb_model = xgb.XGBRegressor(verbosity = 0)
clf = GridSearchCV(xgb_model,grid, cv=5,n_jobs=1)
clf.fit(X_train, y_train)
modelxgb=clf.best_estimator_
ypred=modelxgb.predict(X_test)

In [132]:
SC2_Xgb_4vR2=r2_score(y_test,ypred)
SC2_Xgb_4vMAE=mae(y_test,ypred)
SC2_Xgb_4vMSE=mse(y_test,ypred)
SC2_Xgb_4vRMSE=mse(y_test,ypred,squared=False)# ou bien on utilise np.sqrt(np.mean((ytest-ypred)**2))

In [133]:
print("R2 = ",SC2_Xgb_4vR2)
print("Mae = ",SC2_Xgb_4vMAE)
print("Mse = ",SC2_Xgb_4vMSE)
print("Rmse =",SC2_Xgb_4vRMSE)

R2 =  0.9817682728893045
Mae =  0.14682039215116743
Mse =  0.044767782544858
Rmse = 0.21158398461333977


**RF**

In [134]:
clf = GridSearchCV(estimator=RandomForestRegressor(), param_grid=grid, cv= 5,n_jobs=2)
clf.fit(X_train, y_train)
ypred=clf.predict(X_test)

In [135]:
SC2_RF_4vR2=r2_score(y_test,ypred)
SC2_RF_4vMAE=mae(y_test,ypred)
SC2_RF_4vMSE=mse(y_test,ypred)
SC2_RF_4vRMSE=mse(y_test,ypred,squared=False)# ou bien on utilise np.sqrt(np.mean((ytest-ypred)**2))

In [136]:
print("R2 = ",SC2_RF_4vR2)
print("Mae = ",SC2_RF_4vMAE)
print("Mse = ",SC2_RF_4vMSE)
print("Rmse =",SC2_RF_4vRMSE)

R2 =  0.9668617426657546
Mae =  0.20288935947667444
Mse =  0.08137058487370281
Rmse = 0.2852552977136495


**SVR**

In [137]:
svr = SVR()
clf = GridSearchCV(svr,Svrparam,cv=5,n_jobs=-1)
clf.fit(X_train,y_train)
modelsvr=clf.best_estimator_
ypred=modelsvr.predict(X_test)

In [139]:
SC2_SVR_4vR2=r2_score(y_test,ypred)
SC2_SVR_4vMAE=mae(y_test,ypred)
SC2_SVR_4vMSE=mse(y_test,ypred)
SC2_SVR_4vRMSE=mse(y_test,ypred,squared=False)

In [140]:
print("R2 = ",SC2_SVR_4vR2)
print("Mae = ",SC2_SVR_4vMAE)
print("Mse = ",SC2_SVR_4vMSE)
print("Rmse =",SC2_SVR_4vRMSE)

R2 =  0.9784141861645185
Mae =  0.14841281189744007
Mse =  0.05300370140323787
Rmse = 0.23022532745820531


# Entrainement sur 3 variables

**RF**

In [146]:
SC2_3v_rf_r2={}
SC2_3v_rf_mae={}
SC2_3v_rf_mse={}
SC2_3v_rf_Rmse={}
for a in range(0,len(comb3v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in comb3v[a] :
        for x in i:
            xtr.insert(0,x,X_train[x])
            xts.insert(0,x,X_test[x])
    modelrfc =RandomForestRegressor()

    grids = GridSearchCV(modelrfc,grid,cv=5,n_jobs=-1)

    grids.fit(xtr,y_train)
    modelrfc=grids.best_estimator_
    modelrfc.fit(xtr,y_train)
    ypred=modelrfc.predict(xts)
    scoreR2=r2_score(y_test,ypred)
    scoreMae=mae(y_test,ypred)
    scoreMse=mse(y_test,ypred)
    scoreRmse = mse(y_test,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    SC2_3v_rf_r2[var]=scoreR2
    SC2_3v_rf_mae[var]=scoreMae
    SC2_3v_rf_mse[var]=scoreMse
    SC2_3v_rf_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.9179939308315715
Mae 0.3528003474696463
Mse 0.20136489810321279
Rmse 0.44873700326941257
------
R2 0.8302748248724316
Mae 0.48736828955818284
Mse 0.41675808804978615
Rmse 0.645568035182804
------
R2 0.9666717126119387
Mae 0.2028962375411681
Mse 0.08183720134259612
Rmse 0.2860720212509362
------
R2 0.9493376489178633
Mae 0.25359407336768003
Mse 0.12440078236613226
Rmse 0.35270495086705583
------


In [147]:
SC2_3v_rf_r2['Rayonnement  & Humidité & Vitesse du vent'] = SC2_3v_rf_r2.pop('Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,')
SC2_3v_rf_r2['Température  & Humidité & Vitesse du vent'] = SC2_3v_rf_r2.pop('Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,')
SC2_3v_rf_r2['Température  & Rayonement & Vitesse du vent'] = SC2_3v_rf_r2.pop('Température moy,Température max,Température min,Rayonnement solaire moy,Vitesse du vent  moy,Vitesse du vent  max,')
SC2_3v_rf_r2['Température  & Rayonement &  Humidité'] = SC2_3v_rf_r2.pop('Température moy,Température max,Température min,Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,')

In [148]:
Variables = list(SC2_3v_rf_r2.keys())
ScoresR2 = list(SC2_3v_rf_r2.values())
ScoreMae=list(SC2_3v_rf_mae.values())
ScoreMse=list(SC2_3v_rf_mse.values())
ScoreRmse=list(SC2_3v_rf_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [149]:
print("La meilleur combinaison pour trois variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour trois variables est: Température  & Rayonement & Vitesse du vent
R2 0.9666717126119387
Mae 0.2028962375411681
Mse 0.08183720134259612
Rmse 0.2860720212509362


**xgboost**

In [151]:
SC2_3v_xgb_r2={}
SC2_3v_xgb_mae={}
SC2_3v_xgb_mse={}
SC2_3v_xgb_Rmse={}

for a in range(0,len(comb3v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in comb3v[a] :
        
        for x in i:
            
            xtr.insert(0,x,X_train[x])
            xts.insert(0,x,X_test[x])
    modelxgb =xgb.XGBRegressor(verbosity = 0)

    grids = GridSearchCV(modelxgb,xgparam,cv=5,n_jobs=-1)

    grids.fit(xtr,y_train)
    modelxgb=grids.best_estimator_
    modelxgb.fit(xtr,y_train)
    ypred=modelxgb.predict(xts)
    scoreR2=r2_score(y_test,ypred)
    scoreMae=mae(y_test,ypred)
    scoreMse=mse(y_test,ypred)
    scoreRmse = mse(y_test,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    SC2_3v_xgb_r2[var]=scoreR2
    SC2_3v_xgb_mae[var]=scoreMae
    SC2_3v_xgb_mse[var]=scoreMse
    SC2_3v_xgb_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.9278391095902319
Mae 0.3285816743875005
Mse 0.17719018228463318
Rmse 0.42093964209210943
------
R2 0.8489129196801128
Mae 0.4635382184367271
Mse 0.3709924746038042
Rmse 0.6090915157870812
------
R2 0.9807234439856386
Mae 0.15987023659003008
Mse 0.04733334711654666
Rmse 0.21756228330422223
------
R2 0.9578246996890627
Mae 0.22360441821395968
Mse 0.10356093318095384
Rmse 0.3218088457158284
------


In [152]:
SC2_3v_xgb_r2['Rayonnement  & Humidité & Vitesse du vent'] = SC2_3v_xgb_r2.pop('Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,')
SC2_3v_xgb_r2['Température  & Humidité & Vitesse du vent'] = SC2_3v_xgb_r2.pop('Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,')
SC2_3v_xgb_r2['Température  & Rayonement & Vitesse du vent'] = SC2_3v_xgb_r2.pop('Température moy,Température max,Température min,Rayonnement solaire moy,Vitesse du vent  moy,Vitesse du vent  max,')
SC2_3v_xgb_r2['Température  & Rayonement &  Humidité'] = SC2_3v_xgb_r2.pop('Température moy,Température max,Température min,Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,')

In [153]:
Variables = list(SC2_3v_xgb_r2.keys())
ScoresR2 = list(SC2_3v_xgb_r2.values())
ScoreMae=list(SC2_3v_xgb_mae.values())
ScoreMse=list(SC2_3v_xgb_mse.values())
ScoreRmse=list(SC2_3v_xgb_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [154]:
print("La meilleur combinaison pour trois variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour trois variables est: Température  & Rayonement & Vitesse du vent
R2 0.9807234439856386
Mae 0.15987023659003008
Mse 0.04733334711654666
Rmse 0.21756228330422223


**svr**

In [155]:
SC2_3v_svr_r2={}
SC2_3v_svr_mae={}
SC2_3v_svr_mse={}
SC2_3v_svr_Rmse={}

for a in range(0,len(comb3v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in comb3v[a] :
        
        for x in i:
            xtr.insert(0,x,X_train[x])
            xts.insert(0,x,X_test[x])
    modelsvr = SVR()

    grids = GridSearchCV(modelsvr,Svrparam,cv=3,n_jobs=-1)

    grids.fit(xtr,y_train)
    modelsvr=grids.best_estimator_
    modelsvr.fit(xtr,y_train)
    ypred=modelsvr.predict(xts)
    scoreR2=r2_score(y_test,ypred)
    scoreMae=mae(y_test,ypred)
    scoreMse=mse(y_test,ypred)
    scoreRmse = mse(y_test,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    SC2_3v_svr_r2[var]=scoreR2
    SC2_3v_svr_mae[var]=scoreMae
    SC2_3v_svr_mse[var]=scoreMse
    SC2_3v_svr_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.9258738101696747
Mae 0.31191918824836196
Mse 0.1820159509329266
Rmse 0.426633274526175
------
R2 0.8565156034277398
Mae 0.4446724333905375
Mse 0.3523241778097271
Rmse 0.5935690168882866
------
R2 0.984898419987282
Mae 0.13949067263740425
Mse 0.03708174469639374
Rmse 0.19256620860471274
------
R2 0.9608650188299516
Mae 0.19623294230595562
Mse 0.0960954667805471
Rmse 0.30999268826949306
------


In [156]:
SC2_3v_svr_r2['Rayonnement  & Humidité & Vitesse du vent'] = SC2_3v_svr_r2.pop('Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,')
SC2_3v_svr_r2['Température  & Humidité & Vitesse du vent'] = SC2_3v_svr_r2.pop('Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,')
SC2_3v_svr_r2['Température  & Rayonement & Vitesse du vent'] = SC2_3v_svr_r2.pop('Température moy,Température max,Température min,Rayonnement solaire moy,Vitesse du vent  moy,Vitesse du vent  max,')
SC2_3v_svr_r2['Température  & Rayonement &  Humidité'] = SC2_3v_svr_r2.pop('Température moy,Température max,Température min,Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,')

In [157]:
Variables = list(SC2_3v_svr_r2.keys())
ScoresR2 = list(SC2_3v_svr_r2.values())
ScoreMae=list(SC2_3v_svr_mae.values())
ScoreMse=list(SC2_3v_svr_mse.values())
ScoreRmse=list(SC2_3v_svr_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [158]:
print("La meilleur combinaison pour trois variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour trois variables est: Température  & Rayonement & Vitesse du vent
R2 0.984898419987282
Mae 0.13949067263740425
Mse 0.03708174469639374
Rmse 0.19256620860471274


# Entrainement sur 2 variables

**RF**

In [159]:
SC2_2v_rf_r2={}
SC2_2v_rf_mae={}
SC2_2v_rf_mse={}
SC2_2v_rf_Rmse={}
for a in range(0,len(comb2v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in comb2v[a] :
        for x in i:
            xtr.insert(0,x,X_train[x])
            xts.insert(0,x,X_test[x])
    modelrfc =RandomForestRegressor()

    grids = GridSearchCV(modelrfc,grid,cv=5,n_jobs=-1)

    grids.fit(xtr,y_train)
    modelrfc=grids.best_estimator_
    modelrfc.fit(xtr,y_train)
    ypred=modelrfc.predict(xts)
    scoreR2=r2_score(y_test,ypred)
    scoreMae=mae(y_test,ypred)
    scoreMse=mse(y_test,ypred)
    scoreRmse = mse(y_test,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    SC2_2v_rf_r2[var]=scoreR2
    SC2_2v_rf_mae[var]=scoreMae
    SC2_2v_rf_mse[var]=scoreMse
    SC2_2v_rf_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.5256204872732638
Mae 0.8669165627450133
Mse 1.1648330813940118
Rmse 1.0792743309251878
------
R2 0.875558594829817
Mae 0.4245360057856843
Mse 0.3055643457370062
Rmse 0.5527787493536688
------
R2 0.8256828933933521
Mae 0.4969599806468948
Mse 0.42803351953623725
Rmse 0.6542427069033611
------
R2 0.9095746228005981
Mae 0.35582183913659926
Mse 0.22203840581975184
Rmse 0.4712095137194832
------
R2 0.7989086852868614
Mae 0.5302940030684471
Mse 0.49377725950363643
Rmse 0.7026928628523534
------
R2 0.9550978785583745
Mae 0.23392392444981738
Mse 0.11025660905829632
Rmse 0.33204910639587076
------


In [160]:
SC2_2v_rf_r2['Humidité & Vitesse du vent'] = SC2_2v_rf_r2.pop('Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,')
SC2_2v_rf_r2['Rayonnement & Vitesse du vent'] = SC2_2v_rf_r2.pop('Rayonnement solaire moy,Vitesse du vent  moy,Vitesse du vent  max,')
SC2_2v_rf_r2['Température & Vitesse du vent'] = SC2_2v_rf_r2.pop('Température moy,Température max,Température min,Vitesse du vent  moy,Vitesse du vent  max,')
SC2_2v_rf_r2['Rayonnement & Humidité'] = SC2_2v_rf_r2.pop('Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,')
SC2_2v_rf_r2['Température & Humidité'] = SC2_2v_rf_r2.pop('Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,')
SC2_2v_rf_r2['Température & Rayonnement'] = SC2_2v_rf_r2.pop('Température moy,Température max,Température min,Rayonnement solaire moy,')

In [161]:
Variables = list(SC2_2v_rf_r2.keys())
ScoresR2 = list(SC2_2v_rf_r2.values())
ScoreMae=list(SC2_2v_rf_mae.values())
ScoreMse=list(SC2_2v_rf_mse.values())
ScoreRmse=list(SC2_2v_rf_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [162]:
print("La meilleur combinaison pour deux variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour deux variables est: Température & Rayonnement
R2 0.9550978785583745
Mae 0.23392392444981738
Mse 0.11025660905829632
Rmse 0.33204910639587076


**XGboost**

In [163]:
SC2_2v_xgb_r2={}
SC2_2v_xgb_mae={}
SC2_2v_xgb_mse={}
SC2_2v_xgb_Rmse={}

for a in range(0,len(comb2v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in comb2v[a] :
        
        for x in i:
            
            xtr.insert(0,x,X_train[x])
            xts.insert(0,x,X_test[x])
    modelxgb =xgb.XGBRegressor(verbosity = 0)

    grids = GridSearchCV(modelxgb,xgparam,cv=5,n_jobs=-1)

    grids.fit(xtr,y_train)
    modelxgb=grids.best_estimator_
    modelxgb.fit(xtr,y_train)
    ypred=modelxgb.predict(xts)
    scoreR2=r2_score(y_test,ypred)
    scoreMae=mae(y_test,ypred)
    scoreMse=mse(y_test,ypred)
    scoreRmse = mse(y_test,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    SC2_2v_xgb_r2[var]=scoreR2
    SC2_2v_xgb_mae[var]=scoreMae
    SC2_2v_xgb_mse[var]=scoreMse
    SC2_2v_xgb_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.5327910832492233
Mae 0.8591423073676739
Mse 1.1472257708293165
Rmse 1.071086257417822
------
R2 0.8890544605266372
Mae 0.4019396375888472
Mse 0.27242541287005834
Rmse 0.5219438790426212
------
R2 0.8276869300906031
Mae 0.5053335523648057
Mse 0.4231126319796317
Rmse 0.6504710846606725
------
R2 0.8890363684539833
Mae 0.3991424473607616
Mse 0.2724698377327956
Rmse 0.5219864344336888
------
R2 0.7802614310272826
Mae 0.5555326905932017
Mse 0.5395653638715312
Rmse 0.7345511308762183
------
R2 0.945489797822216
Mae 0.2608965472051292
Mse 0.13384913358755204
Rmse 0.3658539785044739
------


In [164]:
SC2_2v_xgb_r2['Humidité & Vitesse du vent'] = SC2_2v_xgb_r2.pop('Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,')
SC2_2v_xgb_r2['Rayonnement & Vitesse du vent'] = SC2_2v_xgb_r2.pop('Rayonnement solaire moy,Vitesse du vent  moy,Vitesse du vent  max,')
SC2_2v_xgb_r2['Température & Vitesse du vent'] = SC2_2v_xgb_r2.pop('Température moy,Température max,Température min,Vitesse du vent  moy,Vitesse du vent  max,')
SC2_2v_xgb_r2['Rayonnement & Humidité'] = SC2_2v_xgb_r2.pop('Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,')
SC2_2v_xgb_r2['Température & Humidité'] = SC2_2v_xgb_r2.pop('Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,')
SC2_2v_xgb_r2['Température & Rayonnement'] = SC2_2v_xgb_r2.pop('Température moy,Température max,Température min,Rayonnement solaire moy,')

In [165]:
Variables = list(SC2_2v_xgb_r2.keys())
ScoresR2 = list(SC2_2v_xgb_r2.values())
ScoreMae=list(SC2_2v_xgb_mae.values())
ScoreMse=list(SC2_2v_xgb_mse.values())
ScoreRmse=list(SC2_2v_xgb_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [166]:
print("La meilleur combinaison pour deux variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour deux variables est: Température & Rayonnement
R2 0.945489797822216
Mae 0.2608965472051292
Mse 0.13384913358755204
Rmse 0.3658539785044739


**svr**

In [167]:
SC2_2v_svr_r2={}
SC2_2v_svr_mae={}
SC2_2v_svr_mse={}
SC2_2v_svr_Rmse={}

for a in range(0,len(comb2v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in comb2v[a] :
        
        for x in i:
            xtr.insert(0,x,X_train[x])
            xts.insert(0,x,X_test[x])
    modelsvr = SVR()

    grids = GridSearchCV(modelsvr,Svrparam,cv=3,n_jobs=-1)

    grids.fit(xtr,y_train)
    modelsvr=grids.best_estimator_
    modelsvr.fit(xtr,y_train)
    ypred=modelsvr.predict(xts)
    scoreR2=r2_score(y_test,ypred)
    scoreMae=mae(y_test,ypred)
    scoreMse=mse(y_test,ypred)
    scoreRmse = mse(y_test,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    SC2_2v_svr_r2[var]=scoreR2
    SC2_2v_svr_mae[var]=scoreMae
    SC2_2v_svr_mse[var]=scoreMse
    SC2_2v_svr_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.45750990252520707
Mae 0.8919326111177568
Mse 1.332077787750732
Rmse 1.154156743146585
------
R2 0.8971822916360703
Mae 0.3732130154595886
Mse 0.25246762316318094
Rmse 0.502461563866512
------
R2 0.8566999389810341
Mae 0.45429946177231534
Mse 0.3518715441170952
Rmse 0.5931876129160952
------
R2 0.9061038375479206
Mae 0.3420747220795349
Mse 0.2305608764835768
Rmse 0.48016755042753234
------
R2 0.80169029966455
Mae 0.5091645212244282
Mse 0.48694703947961265
Rmse 0.6978159065825403
------
R2 0.9563755442582824
Mae 0.2086143339868164
Mse 0.10711931658615592
Rmse 0.3272908745842999
------


In [168]:
SC2_2v_svr_r2['Humidité & Vitesse du vent'] = SC2_2v_svr_r2.pop('Humidité moy,Humidité max,Humidité min,Vitesse du vent  moy,Vitesse du vent  max,')
SC2_2v_svr_r2['Rayonnement & Vitesse du vent'] = SC2_2v_svr_r2.pop('Rayonnement solaire moy,Vitesse du vent  moy,Vitesse du vent  max,')
SC2_2v_svr_r2['Température & Vitesse du vent'] = SC2_2v_svr_r2.pop('Température moy,Température max,Température min,Vitesse du vent  moy,Vitesse du vent  max,')
SC2_2v_svr_r2['Rayonnement & Humidité'] = SC2_2v_svr_r2.pop('Rayonnement solaire moy,Humidité moy,Humidité max,Humidité min,')
SC2_2v_svr_r2['Température & Humidité'] = SC2_2v_svr_r2.pop('Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,')
SC2_2v_svr_r2['Température & Rayonnement'] = SC2_2v_svr_r2.pop('Température moy,Température max,Température min,Rayonnement solaire moy,')

In [169]:
Variables = list(SC2_2v_svr_r2.keys())
ScoresR2 = list(SC2_2v_svr_r2.values())
ScoreMae=list(SC2_2v_svr_mae.values())
ScoreMse=list(SC2_2v_svr_mse.values())
ScoreRmse=list(SC2_2v_svr_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [170]:
print("La meilleur combinaison pour deux variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour deux variables est: Température & Rayonnement
R2 0.9563755442582824
Mae 0.2086143339868164
Mse 0.10711931658615592
Rmse 0.3272908745842999


# Entrainement sur 1 variable

In [171]:
SC2_1v_rf_r2={}
SC2_1v_rf_mae={}
SC2_1v_rf_mse={}
SC2_1v_rf_Rmse={}
for a in range(0,len(comb1v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in comb1v[a] :
        for x in i:
            xtr.insert(0,x,X_train[x])
            xts.insert(0,x,X_test[x])
    modelrfc =RandomForestRegressor()

    grids = GridSearchCV(modelrfc,grid,cv=5,n_jobs=-1)

    grids.fit(xtr,y_train)
    modelrfc=grids.best_estimator_
    modelrfc.fit(xtr,y_train)
    ypred=modelrfc.predict(xts)
    scoreR2=r2_score(y_test,ypred)
    scoreMae=mae(y_test,ypred)
    scoreMse=mse(y_test,ypred)
    scoreRmse = mse(y_test,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    SC2_1v_rf_r2[var]=scoreR2
    SC2_1v_rf_mae[var]=scoreMae
    SC2_1v_rf_mse[var]=scoreMse
    SC2_1v_rf_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.192178289866189
Mae 1.153183148432453
Mse 1.983596311787167
Rmse 1.4084020419564744
------
R2 0.4447916901440214
Mae 0.9691445821656972
Mse 1.3633072024289619
Rmse 1.1676074693273257
------
R2 0.8606966766404908
Mae 0.4325760120786733
Mse 0.3420576037624027
Rmse 0.5848569087925718
------
R2 0.770278753315683
Mae 0.5806993119605444
Mse 0.5640777066844135
Rmse 0.7510510679603708
------


In [172]:
SC2_1v_rf_r2['Vitesse du vent'] = SC2_1v_rf_r2.pop('Vitesse du vent  moy,Vitesse du vent  max,')
SC2_1v_rf_r2['Humidité'] = SC2_1v_rf_r2.pop('Humidité moy,Humidité max,Humidité min,')
SC2_1v_rf_r2['Rayonnement'] = SC2_1v_rf_r2.pop( 'Rayonnement solaire moy,')
SC2_1v_rf_r2['Température'] = SC2_1v_rf_r2.pop('Température moy,Température max,Température min,')

In [173]:
Variables = list(SC2_1v_rf_r2.keys())
ScoresR2 = list(SC2_1v_rf_r2.values())
ScoreMae=list(SC2_1v_rf_mae.values())
ScoreMse=list(SC2_1v_rf_mse.values())
ScoreRmse=list(SC2_1v_rf_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [174]:
print("La meilleur combinaison pour deux variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour deux variables est: Rayonnement
R2 0.8606966766404908
Mae 0.4325760120786733
Mse 0.3420576037624027
Rmse 0.5848569087925718


**XGBoost**

In [175]:
SC2_1v_xgb_r2={}
SC2_1v_xgb_mae={}
SC2_1v_xgb_mse={}
SC2_1v_xgb_Rmse={}

for a in range(0,len(comb1v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in comb1v[a] :
        
        for x in i:
            
            xtr.insert(0,x,X_train[x])
            xts.insert(0,x,X_test[x])
    modelxgb =xgb.XGBRegressor(verbosity = 0)

    grids = GridSearchCV(modelxgb,xgparam,cv=5,n_jobs=-1)

    grids.fit(xtr,y_train)
    modelxgb=grids.best_estimator_
    modelxgb.fit(xtr,y_train)
    ypred=modelxgb.predict(xts)
    scoreR2=r2_score(y_test,ypred)
    scoreMae=mae(y_test,ypred)
    scoreMse=mse(y_test,ypred)
    scoreRmse = mse(y_test,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    SC2_1v_xgb_r2[var]=scoreR2
    SC2_1v_xgb_mae[var]=scoreMae
    SC2_1v_xgb_mse[var]=scoreMse
    SC2_1v_xgb_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.20051491584741854
Mae 1.153510491312689
Mse 1.9631258288307525
Rmse 1.4011159226954608
------
R2 0.3564812542509691
Mae 1.0464932023909443
Mse 1.5801523957832722
Rmse 1.2570411273237134
------
R2 0.8609214680452094
Mae 0.432313011425316
Mse 0.34150563122226446
Rmse 0.5843848314443698
------
R2 0.7320830817569646
Mae 0.6345006974639407
Mse 0.6578667102227781
Rmse 0.8110898287013456
------


In [176]:
SC2_1v_xgb_r2['Vitesse du vent'] = SC2_1v_xgb_r2.pop('Vitesse du vent  moy,Vitesse du vent  max,')
SC2_1v_xgb_r2['Humidité'] = SC2_1v_xgb_r2.pop('Humidité moy,Humidité max,Humidité min,')
SC2_1v_xgb_r2['Rayonnement'] = SC2_1v_xgb_r2.pop( 'Rayonnement solaire moy,')
SC2_1v_xgb_r2['Température'] = SC2_1v_xgb_r2.pop('Température moy,Température max,Température min,')

In [177]:
Variables = list(SC2_1v_xgb_r2.keys())
ScoresR2 = list(SC2_1v_xgb_r2.values())
ScoreMae=list(SC2_1v_xgb_mae.values())
ScoreMse=list(SC2_1v_xgb_mae.values())
ScoreRmse=list(SC2_1v_xgb_mae.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [178]:
print("La meilleur combinaison pour deux variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour deux variables est: Rayonnement
R2 0.8609214680452094
Mae 0.432313011425316
Mse 0.432313011425316
Rmse 0.432313011425316


**svr**

In [179]:
SC2_1v_svr_r2={}
SC2_1v_svr_mae={}
SC2_1v_svr_mse={}
SC2_1v_svr_Rmse={}

for a in range(0,len(comb1v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in comb1v[a] :
        
        for x in i:
            xtr.insert(0,x,X_train[x])
            xts.insert(0,x,X_test[x])
    modelsvr = SVR()

    grids = GridSearchCV(modelsvr,Svrparam,cv=3,n_jobs=-1)

    grids.fit(xtr,y_train)
    modelsvr=grids.best_estimator_
    modelsvr.fit(xtr,y_train)
    ypred=modelsvr.predict(xts)
    scoreR2=r2_score(y_test,ypred)
    scoreMae=mae(y_test,ypred)
    scoreMse=mse(y_test,ypred)
    scoreRmse = mse(y_test,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    SC2_1v_svr_r2[var]=scoreR2
    SC2_1v_svr_mae[var]=scoreMae
    SC2_1v_svr_mse[var]=scoreMse
    SC2_1v_svr_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.17856420682444074
Mae 1.1319208578691196
Mse 2.0170255258961847
Rmse 1.4202202385180211
------
R2 0.40809974088006074
Mae 0.9367942456881027
Mse 1.4534038342961824
Rmse 1.205571994654895
------
R2 0.86261115299143
Mae 0.4216545419885049
Mse 0.3373566305388706
Rmse 0.5808240960384397
------
R2 0.7915935216676308
Mae 0.5415560091586502
Mse 0.5117395541451385
Rmse 0.7153597375762341
------


In [180]:
SC2_1v_svr_r2['Vitesse du vent'] = SC2_1v_svr_r2.pop('Vitesse du vent  moy,Vitesse du vent  max,')
SC2_1v_svr_r2['Humidité'] = SC2_1v_svr_r2.pop('Humidité moy,Humidité max,Humidité min,')
SC2_1v_svr_r2['Rayonnement'] = SC2_1v_svr_r2.pop( 'Rayonnement solaire moy,')
SC2_1v_svr_r2['Température'] = SC2_1v_svr_r2.pop('Température moy,Température max,Température min,')

In [181]:
Variables = list(SC2_1v_svr_r2.keys())
ScoresR2 = list(SC2_1v_svr_r2.values())
ScoreMae=list(SC2_1v_svr_mae.values())
ScoreMse=list(SC2_1v_svr_mse.values())
ScoreRmse=list(SC2_1v_svr_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [182]:
print("La meilleur combinaison pour deux variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour deux variables est: Rayonnement
R2 0.86261115299143
Mae 0.4216545419885049
Mse 0.3373566305388706
Rmse 0.5808240960384397


# Entrainement sur des données Horraires

In [183]:
#chargement des données néttoyées
EnamMksHr= pd.read_excel(r'EnamHorraire.xlsx')
EnsamMksHr= pd.read_excel(r'EnsamHorraire.xlsx')

In [184]:
Data=EnamMksHr.drop(columns=['Date/heure','ETP quotidien','ET Daily calculé','Unnamed: 0'])

In [185]:
Y=Data['ET Horraire']
X=Data.drop(columns=['ET Horraire'])

In [186]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

# Entrainement sur 4 variables

**XGBoost**

In [188]:
xgb_model = xgb.XGBRegressor(verbosity = 0)
clf = GridSearchCV(xgb_model,grid, cv=5,n_jobs=1)
clf.fit(X_train, y_train)
modelxgb=clf.best_estimator_
ypred=modelxgb.predict(X_test)

In [189]:
HR_Xgb_4vR2=r2_score(y_test,ypred)
HR_Xgb_4vMAE=mae(y_test,ypred)
HR_Xgb_4vMSE=mse(y_test,ypred)
HR_Xgb_4vRMSE=mse(y_test,ypred,squared=False)

In [190]:
print("R2 = ",HR_Xgb_4vR2)
print("Mae = ",HR_Xgb_4vMAE)
print("Mse = ",HR_Xgb_4vMSE)
print("Rmse =",HR_Xgb_4vRMSE)

R2 =  0.9750021795246668
Mae =  0.25511858909377133
Mse =  0.25373951149158436
Rmse = 0.503725631164014


**RF**

In [191]:
clf = GridSearchCV(estimator=RandomForestRegressor(), param_grid=grid, cv= 5,n_jobs=2)
clf.fit(X_train, y_train)
ypred=clf.predict(X_test)

In [192]:
HR_RF_4vR2=r2_score(y_test,ypred)
HR_RF_4vMAE=mae(y_test,ypred)
HR_RF_4vMSE=mse(y_test,ypred)
HR_RF_4vRMSE=mse(y_test,ypred,squared=False)

In [193]:
print("R2 = ",HR_RF_4vR2)
print("Mae = ",HR_RF_4vMAE)
print("Mse = ",HR_RF_4vMSE)
print("Rmse =",HR_RF_4vRMSE)

R2 =  0.9633118101186331
Mae =  0.3277694516124471
Mse =  0.37240220151170766
Rmse = 0.6102476558838286


**svr**

In [195]:
svr = SVR(C= 170, epsilon= 0.2, gamma =0.001)
svr.fit(X_train,y_train)
ypred=svr.predict(X_test)

In [196]:
HR_SVR_4vR2=r2_score(y_test,ypred)
HR_SVR_4vMAE=mae(y_test,ypred)
HR_SVR_4vMSE=mse(y_test,ypred)
HR_SVR_4vRMSE=mse(y_test,ypred,squared=False)

In [197]:
print("R2 = ",HR_SVR_4vR2)
print("Mae = ",HR_SVR_4vMAE)
print("Mse = ",HR_SVR_4vMSE)
print("Rmse =",HR_SVR_4vRMSE)

R2 =  0.9587587709317591
Mae =  0.34583450533625704
Mse =  0.4186176681848702
Rmse = 0.6470066987171541


**meilleur model**

In [198]:
scoresR2=[HR_RF_4vR2,HR_Xgb_4vR2,HR_SVR_4vR2]
scoreMae=[HR_RF_4vMAE,HR_Xgb_4vMAE,HR_SVR_4vMAE]
scoreMse=[HR_RF_4vMSE,HR_Xgb_4vMSE,HR_SVR_4vMSE]
scoreRmse=[HR_RF_4vRMSE,HR_Xgb_4vRMSE,HR_SVR_4vRMSE]
Models=['Rf','Xgb','Svr']

In [199]:
tmp = max(scoresR2)
index = scoresR2.index(tmp)

In [200]:
print("Le meilleur model est:",Models[index])
print("R2=",scoresR2[index])
print("Mae=",scoreMae[index])
print("mse=",scoreMse[index])
print("Rmse=",scoreRmse[index])

Le meilleur model est: Xgb
R2= 0.9750021795246668
Mae= 0.25511858909377133
mse= 0.25373951149158436
Rmse= 0.503725631164014


# Entrainement sur 3 variables

**RF**

In [202]:
temp=['Température min', 'Température max', 'Température moy']
hum=['Humidité min', 'Humidité max', 'Humidité moy']
VitVent=['Vitesse du vent max', 'Vitesse du vent moy']
Ray=['Radiation Solaire moy']

In [203]:
comb = combinations([VitVent, hum,Ray, temp], 3)
co=list(comb)

In [215]:
rfc =RandomForestRegressor(max_depth= 8, max_features= 'auto', n_estimators=500)

In [216]:
HR_3v_rf_r2={}
HR_3v_rf_mae={}
HR_3v_rf_mse={}
HR_3v_rf_Rmse={}
for a in range(0,len(co)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in co[a] :
        for x in i:
            xtr.insert(0,x,X_train[x])
            xts.insert(0,x,X_test[x])
    rfc.fit(xtr,y_train)
    ypred=rfc.predict(xts)
    scoreR2=r2_score(y_test,ypred)
    scoreMae=mae(y_test,ypred)
    scoreMse=mse(y_test,ypred)
    scoreRmse = mse(y_test,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    HR_3v_rf_r2[var]=scoreR2
    HR_3v_rf_mae[var]=scoreMae
    HR_3v_rf_mse[var]=scoreMse
    HR_3v_rf_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.9164800474275702
Mae 0.4631344610401874
Mse 0.847766387731297
Rmse 0.9207423025642392
------
R2 0.7811628006289938
Mae 0.8382795114895678
Mse 2.2212994176583516
Rmse 1.4904024348001956
------
R2 0.9573608673637974
Mae 0.300998233233176
Mse 0.4328070399661799
Rmse 0.657880718646002
------
R2 0.9732508134079548
Mae 0.24736437452890017
Mse 0.27151669263967393
Rmse 0.5210726366253307
------


In [217]:
HR_3v_rf_r2['Rayonnement  & Humidité & Vitesse du vent'] = HR_3v_rf_r2.pop('Radiation Solaire moy,Humidité moy,Humidité max,Humidité min,Vitesse du vent moy,Vitesse du vent max,')
HR_3v_rf_r2['Température  & Humidité & Vitesse du vent'] = HR_3v_rf_r2.pop('Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,Vitesse du vent moy,Vitesse du vent max,')
HR_3v_rf_r2['Température  & Rayonement & Vitesse du vent'] = HR_3v_rf_r2.pop('Température moy,Température max,Température min,Radiation Solaire moy,Vitesse du vent moy,Vitesse du vent max,')
HR_3v_rf_r2['Température  & Rayonement &  Humidité'] = HR_3v_rf_r2.pop('Température moy,Température max,Température min,Radiation Solaire moy,Humidité moy,Humidité max,Humidité min,')

In [218]:
Variables = list(HR_3v_rf_r2.keys())
ScoresR2 = list(HR_3v_rf_r2.values())
ScoreMae=list(HR_3v_rf_mae.values())
ScoreMse=list(HR_3v_rf_mse.values())
ScoreRmse=list(HR_3v_rf_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [219]:
print("La meilleur combinaison pour trois variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour trois variables est: Température  & Rayonement &  Humidité
R2 0.9732508134079548
Mae 0.24736437452890017
Mse 0.27151669263967393
Rmse 0.5210726366253307


**XGBoost**

In [220]:
modelxgb =xgb.XGBRegressor(colsample_bytree=0.4,
 gamma= 0.1,
 learning_rate= 0.15,
 max_depth= 3,min_child_weight= 3,verbosity = 0)

In [221]:
HR_3v_xgb_r2={}
HR_3v_xgb_mae={}
HR_3v_xgb_mse={}
HR_3v_xgb_Rmse={}

for a in range(0,len(co)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in co[a] :
        for x in i:
            xtr.insert(0,x,X_train[x])
            xts.insert(0,x,X_test[x])
    modelxgb.fit(xtr,y_train)
    ypred=modelxgb.predict(xts)
    scoreR2=r2_score(y_test,ypred)
    scoreMae=mae(y_test,ypred)
    scoreMse=mse(y_test,ypred)
    scoreRmse = mse(y_test,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    HR_3v_xgb_r2[var]=scoreR2
    HR_3v_xgb_mae[var]=scoreMae
    HR_3v_xgb_mse[var]=scoreMse
    HR_3v_xgb_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.9114710623205101
Mae 0.5228686605349239
Mse 0.8986099176857651
Rmse 0.947950377227503
------
R2 0.7696397741444287
Mae 0.892145681625145
Mse 2.3382634991463056
Rmse 1.529138155676689
------
R2 0.9518516276496902
Mae 0.3780213813143551
Mse 0.4887283869942935
Rmse 0.6990911149444639
------
R2 0.9655747399951792
Mae 0.3535644956049358
Mse 0.34943241012605125
Rmse 0.5911280826741792
------


In [222]:
HR_3v_xgb_r2['Rayonnement  & Humidité & Vitesse du vent'] = HR_3v_xgb_r2.pop('Radiation Solaire moy,Humidité moy,Humidité max,Humidité min,Vitesse du vent moy,Vitesse du vent max,')
HR_3v_xgb_r2['Température  & Humidité & Vitesse du vent'] = HR_3v_xgb_r2.pop('Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,Vitesse du vent moy,Vitesse du vent max,')
HR_3v_xgb_r2['Température  & Rayonement & Vitesse du vent'] = HR_3v_xgb_r2.pop('Température moy,Température max,Température min,Radiation Solaire moy,Vitesse du vent moy,Vitesse du vent max,')
HR_3v_xgb_r2['Température  & Rayonement &  Humidité'] = HR_3v_xgb_r2.pop('Température moy,Température max,Température min,Radiation Solaire moy,Humidité moy,Humidité max,Humidité min,')

In [223]:
Variables = list(HR_3v_xgb_r2.keys())
ScoresR2 = list(HR_3v_xgb_r2.values())
ScoreMae=list(HR_3v_xgb_mae.values())
ScoreMse=list(HR_3v_xgb_mse.values())
ScoreRmse=list(HR_3v_xgb_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [224]:
print("La meilleur combinaison pour trois variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour trois variables est: Température  & Rayonement &  Humidité
R2 0.9655747399951792
Mae 0.3535644956049358
Mse 0.34943241012605125
Rmse 0.5911280826741792


**SVR**

In [225]:
HR_3v_svr_r2={}
HR_3v_svr_mae={}
HR_3v_svr_mse={}
HR_3v_svr_Rmse={}

for a in range(0,len(co)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in co[a] :
        for x in i:
            xtr.insert(0,x,X_train[x])
            xts.insert(0,x,X_test[x])
    svr.fit(xtr,y_train)
    ypred=svr.predict(xts)
    scoreR2=r2_score(y_test,ypred)
    scoreMae=mae(y_test,ypred)
    scoreMse=mse(y_test,ypred)
    scoreRmse = mse(y_test,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    HR_3v_svr_r2[var]=scoreR2
    HR_3v_svr_mae[var]=scoreMae
    HR_3v_svr_mse[var]=scoreMse
    HR_3v_svr_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.9008167012417054
Mae 0.50631284011597
Mse 1.0067566410394468
Rmse 1.0033726331923982
------
R2 0.774404960020469
Mae 0.853693156342305
Mse 2.2898946448477413
Rmse 1.513239784319637
------
R2 0.9556266710589564
Mae 0.3344293527622547
Mse 0.4504099395331673
Rmse 0.6711258745817861
------
R2 0.9699861963475694
Mae 0.3256892863847125
Mse 0.3046540751137447
Rmse 0.5519547763302214
------


In [226]:
HR_3v_svr_r2['Rayonnement  & Humidité & Vitesse du vent'] = HR_3v_svr_r2.pop('Radiation Solaire moy,Humidité moy,Humidité max,Humidité min,Vitesse du vent moy,Vitesse du vent max,')
HR_3v_svr_r2['Température  & Humidité & Vitesse du vent'] = HR_3v_svr_r2.pop('Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,Vitesse du vent moy,Vitesse du vent max,')
HR_3v_svr_r2['Température  & Rayonement & Vitesse du vent'] = HR_3v_svr_r2.pop('Température moy,Température max,Température min,Radiation Solaire moy,Vitesse du vent moy,Vitesse du vent max,')
HR_3v_svr_r2['Température  & Rayonement &  Humidité'] = HR_3v_svr_r2.pop('Température moy,Température max,Température min,Radiation Solaire moy,Humidité moy,Humidité max,Humidité min,')

In [227]:
Variables = list(HR_3v_svr_r2.keys())
ScoresR2 = list(HR_3v_svr_r2.values())
ScoreMae=list(HR_3v_svr_mae.values())
ScoreMse=list(HR_3v_svr_mse.values())
ScoreRmse=list(HR_3v_svr_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [228]:
print("La meilleur combinaison pour trois variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour trois variables est: Température  & Rayonement &  Humidité
R2 0.9699861963475694
Mae 0.3256892863847125
Mse 0.3046540751137447
Rmse 0.5519547763302214


# Entrainement sur 2 variables

**RF**

In [230]:
comb = combinations([VitVent, hum,Ray, temp], 2)
co2v=list(comb)

In [231]:
HR_2v_rf_r2={}
HR_2v_rf_mae={}
HR_2v_rf_mse={}
HR_2v_rf_Rmse={}
for a in range(0,len(co2v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in co2v[a] :
        for x in i:
            xtr.insert(0,x,X_train[x])
            xts.insert(0,x,X_test[x])
    rfc.fit(xtr,y_train)
    ypred=rfc.predict(xts)
    scoreR2=r2_score(y_test,ypred)
    scoreMae=mae(y_test,ypred)
    scoreMse=mse(y_test,ypred)
    scoreRmse = mse(y_test,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    HR_2v_rf_r2[var]=scoreR2
    HR_2v_rf_mae[var]=scoreMae
    HR_2v_rf_mse[var]=scoreMse
    HR_2v_rf_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.534298281249167
Mae 1.2725567045920885
Mse 4.727089176963643
Rmse 2.174187015176855
------
R2 0.90091903357954
Mae 0.5015679314739443
Mse 1.0057179201862614
Rmse 1.0028548849092083
------
R2 0.7352875647198345
Mae 0.9299218304806156
Mse 2.686954411886248
Rmse 1.6391932198146282
------
R2 0.9110498284429855
Mae 0.47956206630421294
Mse 0.9028856375795159
Rmse 0.950202945469817
------
R2 0.7431077794321718
Mae 0.9257160221453106
Mse 2.6075755931277973
Rmse 1.6147989327243801
------
R2 0.9548747857583088
Mae 0.31389026340555193
Mse 0.45804192525257476
Rmse 0.6767879470355355
------


In [232]:
HR_2v_rf_r2

{'Humidité moy,Humidité max,Humidité min,Vitesse du vent moy,Vitesse du vent max,': 0.534298281249167,
 'Radiation Solaire moy,Vitesse du vent moy,Vitesse du vent max,': 0.90091903357954,
 'Température moy,Température max,Température min,Vitesse du vent moy,Vitesse du vent max,': 0.7352875647198345,
 'Radiation Solaire moy,Humidité moy,Humidité max,Humidité min,': 0.9110498284429855,
 'Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,': 0.7431077794321718,
 'Température moy,Température max,Température min,Radiation Solaire moy,': 0.9548747857583088}

In [234]:
HR_2v_rf_r2['Humidité & Vitesse du vent'] = HR_2v_rf_r2.pop('Humidité moy,Humidité max,Humidité min,Vitesse du vent moy,Vitesse du vent max,')
HR_2v_rf_r2['Rayonnement & Vitesse du vent'] = HR_2v_rf_r2.pop('Radiation Solaire moy,Vitesse du vent moy,Vitesse du vent max,')
HR_2v_rf_r2['Température & Vitesse du vent'] = HR_2v_rf_r2.pop('Température moy,Température max,Température min,Vitesse du vent moy,Vitesse du vent max,')
HR_2v_rf_r2['Rayonnement & Humidité'] = HR_2v_rf_r2.pop('Radiation Solaire moy,Humidité moy,Humidité max,Humidité min,')
HR_2v_rf_r2['Température & Humidité'] = HR_2v_rf_r2.pop('Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,')
HR_2v_rf_r2['Température & Rayonnement'] = HR_2v_rf_r2.pop('Température moy,Température max,Température min,Radiation Solaire moy,')

In [235]:
Variables = list(HR_2v_rf_r2.keys())
ScoresR2 = list(HR_2v_rf_r2.values())
ScoreMae=list(HR_2v_rf_mae.values())
ScoreMse=list(HR_2v_rf_mse.values())
ScoreRmse=list(HR_2v_rf_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [236]:
print("La meilleur combinaison pour deux variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour deux variables est: Température & Rayonnement
R2 0.9548747857583088
Mae 0.31389026340555193
Mse 0.45804192525257476
Rmse 0.6767879470355355


**XGBoost**

In [237]:
HR_2v_xgb_r2={}
HR_2v_xgb_mae={}
HR_2v_xgb_mse={}
HR_2v_xgb_Rmse={}

for a in range(0,len(co2v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in co2v[a] :
        for x in i:
            xtr.insert(0,x,X_train[x])
            xts.insert(0,x,X_test[x])
    modelxgb.fit(xtr,y_train)
    ypred=modelxgb.predict(xts)
    scoreR2=r2_score(y_test,ypred)
    scoreMae=mae(y_test,ypred)
    scoreMse=mse(y_test,ypred)
    scoreRmse = mse(y_test,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    HR_2v_xgb_r2[var]=scoreR2
    HR_2v_xgb_mae[var]=scoreMae
    HR_2v_xgb_mse[var]=scoreMse
    HR_2v_xgb_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.5273302293091418
Mae 1.313043011809196
Mse 4.797818146997437
Rmse 2.190392235878642
------
R2 0.8908453841453422
Mae 0.608091136362688
Mse 1.1079701500913868
Rmse 1.0526016103404872
------
R2 0.7284831865409629
Mae 0.9646797188721857
Mse 2.756022017072651
Rmse 1.6601271087096467
------
R2 0.8823832196015812
Mae 0.671301330345039
Mse 1.1938650583940558
Rmse 1.0926413219323419
------
R2 0.7218308072576163
Mae 0.9905819309926257
Mse 2.8235467627311257
Rmse 1.6803412637708823
------
R2 0.9252394519119148
Mae 0.5270228931199515
Mse 0.7588543557000252
Rmse 0.8711224688297422
------


In [238]:
HR_2v_xgb_r2['Humidité & Vitesse du vent'] = HR_2v_xgb_r2.pop('Humidité moy,Humidité max,Humidité min,Vitesse du vent moy,Vitesse du vent max,')
HR_2v_xgb_r2['Rayonnement & Vitesse du vent'] = HR_2v_xgb_r2.pop('Radiation Solaire moy,Vitesse du vent moy,Vitesse du vent max,')
HR_2v_xgb_r2['Température & Vitesse du vent'] = HR_2v_xgb_r2.pop('Température moy,Température max,Température min,Vitesse du vent moy,Vitesse du vent max,')
HR_2v_xgb_r2['Rayonnement & Humidité'] = HR_2v_xgb_r2.pop('Radiation Solaire moy,Humidité moy,Humidité max,Humidité min,')
HR_2v_xgb_r2['Température & Humidité'] = HR_2v_xgb_r2.pop('Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,')
HR_2v_xgb_r2['Température & Rayonnement'] = HR_2v_xgb_r2.pop('Température moy,Température max,Température min,Radiation Solaire moy,')

In [240]:
Variables = list(HR_2v_xgb_r2.keys())
ScoresR2 = list(HR_2v_xgb_r2.values())
ScoreMae=list(HR_2v_xgb_mae.values())
ScoreMse=list(HR_2v_xgb_mse.values())
ScoreRmse=list(HR_2v_xgb_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [241]:
print("La meilleur combinaison pour deux variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour deux variables est: Température & Rayonnement
R2 0.9252394519119148
Mae 0.5270228931199515
Mse 0.7588543557000252
Rmse 0.8711224688297422


**svr**

In [243]:
HR_2v_svr_r2={}
HR_2v_svr_mae={}
HR_2v_svr_mse={}
HR_2v_svr_Rmse={}

for a in range(0,len(co2v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in co2v[a] :
        for x in i:
            xtr.insert(0,x,X_train[x])
            xts.insert(0,x,X_test[x])
    svr.fit(xtr,y_train)
    ypred=svr.predict(xts)
    scoreR2=r2_score(y_test,ypred)
    scoreMae=mae(y_test,ypred)
    scoreMse=mse(y_test,ypred)
    scoreRmse = mse(y_test,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    HR_2v_svr_r2[var]=scoreR2
    HR_2v_svr_mae[var]=scoreMae
    HR_2v_svr_mse[var]=scoreMse
    HR_2v_svr_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.47953948180281447
Mae 1.3355461134848325
Mse 5.282916475391267
Rmse 2.2984595875044804
------
R2 0.8916209958704451
Mae 0.610947107797535
Mse 1.1000973301217836
Rmse 1.0488552474587634
------
R2 0.7082153664212112
Mae 0.9550914153178536
Mse 2.961749823673221
Rmse 1.7209735104507626
------
R2 0.8974601460978004
Mae 0.5277113234424426
Mse 1.0408272378480539
Rmse 1.0202094088215683
------
R2 0.7421656940031698
Mae 0.9145629548670912
Mse 2.6171381986667153
Rmse 1.6177571507079531
------
R2 0.9528846109924181
Mae 0.3805332440451851
Mse 0.4782431253283283
Rmse 0.6915512456270528
------


In [244]:
HR_2v_svr_r2['Humidité & Vitesse du vent'] = HR_2v_svr_r2.pop('Humidité moy,Humidité max,Humidité min,Vitesse du vent moy,Vitesse du vent max,')
HR_2v_svr_r2['Rayonnement & Vitesse du vent'] = HR_2v_svr_r2.pop('Radiation Solaire moy,Vitesse du vent moy,Vitesse du vent max,')
HR_2v_svr_r2['Température & Vitesse du vent'] = HR_2v_svr_r2.pop('Température moy,Température max,Température min,Vitesse du vent moy,Vitesse du vent max,')
HR_2v_svr_r2['Rayonnement & Humidité'] = HR_2v_svr_r2.pop('Radiation Solaire moy,Humidité moy,Humidité max,Humidité min,')
HR_2v_svr_r2['Température & Humidité'] = HR_2v_svr_r2.pop('Température moy,Température max,Température min,Humidité moy,Humidité max,Humidité min,')
HR_2v_svr_r2['Température & Rayonnement'] = HR_2v_svr_r2.pop('Température moy,Température max,Température min,Radiation Solaire moy,')

In [245]:
Variables = list(HR_2v_svr_r2.keys())
ScoresR2 = list(HR_2v_svr_r2.values())
ScoreMae=list(HR_2v_svr_mae.values())
ScoreMse=list(HR_2v_svr_mse.values())
ScoreRmse=list(HR_2v_svr_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [246]:
print("La meilleur combinaison pour deux variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour deux variables est: Température & Rayonnement
R2 0.9528846109924181
Mae 0.3805332440451851
Mse 0.4782431253283283
Rmse 0.6915512456270528


# Entrainement sur 1 variable

In [247]:
comb = combinations([VitVent, hum,Ray, temp], 1)
co1v=list(comb)

**RF**

In [248]:
HR_1v_rf_r2={}
HR_1v_rf_mae={}
HR_1v_rf_mse={}
HR_1v_rf_Rmse={}
for a in range(0,len(co1v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in co1v[a] :
        for x in i:
            xtr.insert(0,x,X_train[x])
            xts.insert(0,x,X_test[x])
    rfc.fit(xtr,y_train)
    ypred=rfc.predict(xts)
    scoreR2=r2_score(y_test,ypred)
    scoreMae=mae(y_test,ypred)
    scoreMse=mse(y_test,ypred)
    scoreRmse = mse(y_test,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    HR_1v_rf_r2[var]=scoreR2
    HR_1v_rf_mae[var]=scoreMae
    HR_1v_rf_mse[var]=scoreMse
    HR_1v_rf_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.36545909762328255
Mae 1.6211331846228623
Mse 6.440885466369905
Rmse 2.5378899634085608
------
R2 0.42957486956801905
Mae 1.47037199414399
Mse 5.790080542467976
Rmse 2.4062586191986877
------
R2 0.8913148771836383
Mae 0.5289197887091168
Mse 1.103204577256606
Rmse 1.0503354593921914
------
R2 0.6566883199701967
Mae 1.1157963430052869
Mse 3.4847733251815187
Rmse 1.8667547576426633
------


In [249]:
HR_1v_rf_r2

{'Vitesse du vent moy,Vitesse du vent max,': 0.36545909762328255,
 'Humidité moy,Humidité max,Humidité min,': 0.42957486956801905,
 'Radiation Solaire moy,': 0.8913148771836383,
 'Température moy,Température max,Température min,': 0.6566883199701967}

In [251]:
HR_1v_rf_r2['Vitesse du vent'] = HR_1v_rf_r2.pop('Vitesse du vent moy,Vitesse du vent max,')
HR_1v_rf_r2['Humidité'] = HR_1v_rf_r2.pop('Humidité moy,Humidité max,Humidité min,')
HR_1v_rf_r2['Rayonnement'] = HR_1v_rf_r2.pop( 'Radiation Solaire moy,')
HR_1v_rf_r2['Température'] = HR_1v_rf_r2.pop('Température moy,Température max,Température min,')

In [252]:
Variables = list(HR_1v_rf_r2.keys())
ScoresR2 = list(HR_1v_rf_r2.values())
ScoreMae=list(HR_1v_rf_mae.values())
ScoreMse=list(HR_1v_rf_mse.values())
ScoreRmse=list(HR_1v_rf_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [253]:
print("La meilleur combinaison pour deux variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour deux variables est: Rayonnement
R2 0.8913148771836383
Mae 0.5289197887091168
Mse 1.103204577256606
Rmse 1.0503354593921914


**XGBoost**

In [254]:
HR_1v_xgb_r2={}
HR_1v_xgb_mae={}
HR_1v_xgb_mse={}
HR_1v_xgb_Rmse={}

for a in range(0,len(co1v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in co1v[a] :
        for x in i:
            xtr.insert(0,x,X_train[x])
            xts.insert(0,x,X_test[x])
    modelxgb.fit(xtr,y_train)
    ypred=modelxgb.predict(xts)
    scoreR2=r2_score(y_test,ypred)
    scoreMae=mae(y_test,ypred)
    scoreMse=mse(y_test,ypred)
    scoreRmse = mse(y_test,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    HR_1v_xgb_r2[var]=scoreR2
    HR_1v_xgb_mae[var]=scoreMae
    HR_1v_xgb_mse[var]=scoreMse
    HR_1v_xgb_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.35400844762923267
Mae 1.6462760604995545
Mse 6.557114892796035
Rmse 2.560686410475917
------
R2 0.36880833477138963
Mae 1.5869999820304608
Mse 6.406889150624348
Rmse 2.531183349863132
------
R2 0.8926120715734029
Mae 0.5269565615189122
Mse 1.0900374505027648
Rmse 1.0440485862749707
------
R2 0.624421762288808
Mae 1.2194298318339267
Mse 3.8122939020921938
Rmse 1.952509641997241
------


In [256]:
HR_1v_xgb_r2['Vitesse du vent'] = HR_1v_xgb_r2.pop('Vitesse du vent moy,Vitesse du vent max,')
HR_1v_xgb_r2['Humidité'] = HR_1v_xgb_r2.pop('Humidité moy,Humidité max,Humidité min,')
HR_1v_xgb_r2['Rayonnement'] = HR_1v_xgb_r2.pop( 'Radiation Solaire moy,')
HR_1v_xgb_r2['Température'] = HR_1v_xgb_r2.pop('Température moy,Température max,Température min,')

In [257]:
Variables = list(HR_1v_xgb_r2.keys())
ScoresR2 = list(HR_1v_xgb_r2.values())
ScoreMae=list(HR_1v_xgb_mae.values())
ScoreMse=list(HR_1v_xgb_mae.values())
ScoreRmse=list(HR_1v_xgb_mae.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [258]:
print("La meilleur combinaison pour deux variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour deux variables est: Rayonnement
R2 0.8926120715734029
Mae 0.5269565615189122
Mse 0.5269565615189122
Rmse 0.5269565615189122


**svr**

In [271]:
HR_1v_svr_r2={}
HR_1v_svr_mae={}
HR_1v_svr_mse={}
HR_1v_svr_Rmse={}

for a in range(0,len(co1v)):
    xtr=pd.DataFrame()
    xts=pd.DataFrame()
    for i in co1v[a] :
        for x in i:
            xtr.insert(0,x,X_train[x])
            xts.insert(0,x,X_test[x])
    svr.fit(xtr,y_train)
    ypred=svr.predict(xts)
    scoreR2=r2_score(y_test,ypred)
    scoreMae=mae(y_test,ypred)
    scoreMse=mse(y_test,ypred)
    scoreRmse = mse(y_test,ypred,squared=False)
    var=''
    for col in xtr.columns:
        var+=col+','
    HR_1v_svr_r2[var]=scoreR2
    HR_1v_svr_mae[var]=scoreMae
    HR_1v_svr_mse[var]=scoreMse
    HR_1v_svr_Rmse[var]=scoreRmse
    print("R2",scoreR2)
    print("Mae",scoreMae)
    print("Mse",scoreMse)
    print("Rmse",scoreRmse)
    print("------")

R2 0.2419544049904686
Mae 1.5833023079185309
Mse 7.69451557410236
Rmse 2.7738989841200707
------
R2 0.38159753490776605
Mae 1.454342926417833
Mse 6.277072817309434
Rmse 2.5054087126274296
------
R2 0.8866963210557128
Mae 0.6324276508187863
Mse 1.15008507137219
Rmse 1.0724201934746427
------
R2 0.6093357695594293
Mae 1.1046722344380897
Mse 3.965423749124073
Rmse 1.9913371761517618
------


In [272]:
HR_1v_svr_r2['Vitesse du vent'] = HR_1v_svr_r2.pop('Vitesse du vent moy,Vitesse du vent max,')
HR_1v_svr_r2['Humidité'] = HR_1v_svr_r2.pop('Humidité moy,Humidité max,Humidité min,')
HR_1v_svr_r2['Rayonnement'] = HR_1v_svr_r2.pop( 'Radiation Solaire moy,')
HR_1v_svr_r2['Température'] = HR_1v_svr_r2.pop('Température moy,Température max,Température min,')

In [273]:
Variables = list(HR_1v_svr_r2.keys())
ScoresR2 = list(HR_1v_svr_r2.values())
ScoreMae=list(HR_1v_svr_mae.values())
ScoreMse=list(HR_1v_svr_mse.values())
ScoreRmse=list(HR_1v_svr_Rmse.values())
tmp = max(ScoresR2)
index = ScoresR2.index(tmp)

In [274]:
print("La meilleur combinaison pour deux variables est:",Variables[index])
print("R2",ScoresR2[index])
print("Mae",ScoreMae[index])
print("Mse",ScoreMse[index])
print("Rmse",ScoreRmse[index])

La meilleur combinaison pour deux variables est: Rayonnement
R2 0.8866963210557128
Mae 0.6324276508187863
Mse 1.15008507137219
Rmse 1.0724201934746427
